In [4]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Preprocess and train self-supervised rotation classifier

In [5]:
from fastai.vision import *
from repr.dataset.rotation_dataset import label_and_folder, init_databunch, rotation_databunch

## Generate dataset

In [6]:
data_root:str = ! pwd ~/data

In [7]:
data_root

['/home/ubuntu/git/repr-vec']

In [8]:
path = Path.home() / 'data' / 'label_engine'
src_root = path / 'cropped_base_data'
dst_root_224 = path / 'rotate_dst_224'
dst_root_400 = path / 'rotate_dst_400'
dst_root_512 = path / 'rotate_dst_512'


In [9]:
dst_root_512

PosixPath('/home/ubuntu/data/label_engine/rotate_dst_512')

In [10]:
dst_root_512.mkdir(exist_ok=True)

In [ ]:
label_and_folder(src_root, dst_root_512, h=512, w=512, test='test', verbose=True)

train and validation directories src_dirs = [PosixPath('/home/ubuntu/data/label_engine/cropped_base_data/train'), PosixPath('/home/ubuntu/data/label_engine/cropped_base_data/valid')], dst_dirs = [PosixPath('/home/ubuntu/data/label_engine/rotate_dst_512/train'), PosixPath('/home/ubuntu/data/label_engine/rotate_dst_512/valid')]


In [ ]:
def init_transforms(size):
    return rand_resize_crop(size) + [rand_crop(p=1.), crop_pad(), brightness(change=(0.1, 0.9), p=1.0), 
            contrast(scale=(0.5, 2.), p=1.), jitter(magnitude=-0.2, p=1.), symmetric_warp(magnitude=(-0.2,0.2), p=1.),
            zoom(scale=1.78, p=1.), cutout(n_holes=(1,4), length=(10, 160), p=1.), squish(scale=(0.68, 1.38))]

In [ ]:
xtra_tfms = [brightness(change=(0.1, 0.9), p=1.0), contrast(scale=(0.5, 2.), p=1.), 
             cutout(n_holes=(1, 4), length=(5, 80), p=1.), squish(scale=(0.68, 1.38)),
             brightness(change=(0.1, 0.9), p=1.0)] #+ rand_resize_crop(224)

In [ ]:
tfms = get_transforms(do_flip=False, flip_vert=False, max_rotate=5.0, max_zoom=1.3, max_lighting=0.75, max_warp=0.4,
                      p_lighting=0.85, p_affine=0.75, xtra_tfms=xtra_tfms)

In [ ]:
data = init_databunch(dst_root_512, bs=64, size=224, ds_tfms=tfms, num_workers=1)

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=[accuracy, error_rate])

## Find learning rate for training

In [ ]:
@gpu_mem_restore
lr_find(learn)

learn.recorder.plot()

In [ ]:
lr=1e-02

## Train for few epochs 

In [ ]:
@gpu_mem_restore
learn.fit_one_cycle(18, slice(lr), pct_start=0.9)

In [ ]:
learn.save('a-rot1')

In [ ]:
learn.show_results(rows=8, figsize=(40, 82))

In [ ]:
mdl = learn.load('a-rot1')

In [ ]:
learn.unfreeze()

In [ ]:
lrs = slice(lr / 400, lr / 4)

In [ ]:
@gpu_mem_restore
learn.fit_one_cycle(12, lrs, pct_start=0.8)

In [ ]:
learn.show_results(rows=8, figsize=(40, 82), ds_type=DatasetType.Valid)

In [ ]:
learn.save('a-rot1-fl')

#### Train on 448 size

In [ ]:
md = learn.load('a-rot1-fl')

In [ ]:
data = init_databunch(dst_root_224, bs=64, size=448, ds_tfms=tfms)

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=[accuracy, error_rate])

In [ ]:
md = learn.load('a-rot1-fl')

In [ ]:
@gpu_mem_restore
lr_find(learn)

learn.recorder.plot()

In [ ]:
lr=1e-02

In [ ]:
@gpu_mem_restore
learn.fit_one_cycle(12, slice(lr), pct_start=0.9)

In [ ]:
learn.save('a-rot2')

In [ ]:
learn.show_results(rows=8, figsize=(40, 82))

In [ ]:
mdl = learn.load('a-rot2')

In [ ]:
learn.unfreeze()

In [ ]:
lrs = slice(lr / 400, lr / 4)

In [ ]:
@gpu_mem_restore
learn.fit_one_cycle(12, lrs, pct_start=0.8)

In [ ]:
learn.show_results(rows=8, figsize=(40, 82), ds_type=DatasetType.Valid)

In [ ]:
learn.save('a-rot2-fl')

#### Train on 512 size

In [ ]:
md = learn.load('a-rot2-fl')

In [ ]:
data = init_databunch(dst_root_224, bs=64, size=512, ds_tfms=tfms)

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=[accuracy, error_rate])

In [ ]:
md = learn.load('a-rot2-fl')

In [ ]:
@gpu_mem_restore
lr_find(learn)

learn.recorder.plot()

In [ ]:
lr=1e-03

In [ ]:
@gpu_mem_restore
learn.fit_one_cycle(12, slice(lr), pct_start=0.9)

In [ ]:
learn.save('a-rot3')

In [ ]:
learn.show_results(rows=8, figsize=(40, 82))

In [ ]:
mdl = learn.load('a-rot3')

In [ ]:
learn.unfreeze()

In [ ]:
lrs = slice(lr / 400, lr / 4)

In [ ]:
@gpu_mem_restore
learn.fit_one_cycle(12, lrs, pct_start=0.8)

In [ ]:
learn.show_results(rows=8, figsize=(40, 82), ds_type=DatasetType.Valid)

In [ ]:
learn.save('a-rot3-fl')

In [ ]:
md = learn.load('a-rot3-fl')

In [ ]:
@gpu_mem_restore
learn.fit_one_cycle(12, lrs, pct_start=0.8)

In [ ]:
learn.save('a-rot3-1-fl')

In [ ]:
learn.show_results(rows=12, figsize=(40, 82), ds_type=DatasetType.Valid)

In [ ]:
md = learn.load('a-rot3-1-fl')

In [ ]:
@gpu_mem_restore
learn.fit_one_cycle(18, lrs, pct_start=0.8)

In [ ]:
learn.save('a-rot3-2-fl')

In [ ]:
learn.show_results(rows=12, figsize=(40, 82), ds_type=DatasetType.Valid)

In [ ]:
md = learn.load('a-rot3-2-fl')

In [ ]:
@gpu_mem_restore
learn.fit_one_cycle(18, lrs, pct_start=0.8)

In [ ]:
learn.save('a-rot3-3-fl')

In [ ]:
learn.show_results(rows=12, figsize=(40, 82), ds_type=DatasetType.Valid)

In [ ]:
md = learn.load('a-rot3-3-fl')

In [ ]:
@gpu_mem_restore
learn.fit_one_cycle(18, lrs, pct_start=0.8)

In [ ]:
learn.save('a-rot3-4-fl')

## Make a few forward calls

In [ ]:
md = learn.load('a-rot3-4-fl')

In [ ]:
learn.show_results(rows=32, figsize=(40, 82), ds_type=DatasetType.Valid)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

losses,idxs = interp.top_losses()

len(data.valid_ds)==len(losses)==len(idxs)

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [ ]:
interp.most_confused(min_val=2)